# Generation Task

## Load Package

In [83]:
import os
import json
from tqdm import tqdm
from openai import OpenAI
from _api_key import get_deepseek_api_key
import requests

## Basic Function

In [84]:
def load_json_file(file_path):
    """
    Load json file
    """
    with open(file_path,'r',encoding='utf-8') as f:
        file = json.load(f)
        f.close()
    return file

def save_json_file(file, file_path, sort_keys:bool=False):
    """
    Save json file
    """
    with open(file_path,'w',encoding='utf-8') as f:
        json.dump(file, f, indent=4, ensure_ascii=False, sort_keys=sort_keys)
        f.close()

In [85]:
def splitDataset(file, puntype='hom'):
    """
    Enter path or json file to separate the pun part from the non-pun part of the dataset
    """
    if isinstance(file,str):
        dataset = load_json_file(file)
    else:
        dataset = file
    punDataset = dict()
    nonpunDataset = dict()
    for ID in dataset:
        data = dataset[ID]
        if puntype in ID:
            if data.get('pun_word', False):
                punDataset[ID] = data
            else:
                nonpunDataset[ID] = data
    return punDataset, nonpunDataset

In [86]:
def add_human_pun(pun_dataset, save:bool=False, path_gen='pun_generation.json'):
    """
    Add human-written puns
    """
    path_gen = './results/' + path_gen
    if os.path.exists(path_gen):
        pun_generation = load_json_file(path_gen)
    else:
        pun_generation = dict()
    for ID in pun_dataset:
        data = pun_dataset[ID]
        human_text = data['human_text']
        if ID not in pun_generation:
            pun_generation[ID] = {'human_text':human_text}
        else:
            pun_generation[ID].update({'human_text':human_text})
    if save:
        save_json_file(pun_generation, path_gen)

## Function of Generation

In [ ]:
def call_llm_to_generate(model, dataset, model_name:str=None, method:int=0, examples:dict=None,
                         save:bool=False, path_gen='pun_generation.json', batch_size:int=1):
    """
    Try generating puns with llm  \n
    method=0: Generate non-pun sentence with only a single layer of meaning \n
    method=1: Given pun pair, generate pun directly \n
    method=2: Given specific contextual word set, generate pun directly  \n
    """
    def parse_output(ID, output:str):
        # Parse output and get the result
        try:
            output = output[output.index('{'): output.index('}')+1]
        except:
            output = output
        try:
            output = eval(output)
            sentence = output['Sentence']
        except:
            # print(ID, output)
            try:
                sentence = output.split('Sentence')[-1]
            except:
                sentence = 'No correctly parsed result.'
        return sentence

    def get_punchline_elements(data:dict):
        # Get pun word/sense, alternative word/sense, etc. from data
        pun_word = data['pun_word']
        pun_sense = data['pun_sense']
        alter_word = data['alter_word']
        alter_sense = data['alter_sense']
        context_words = data['human_keywords']
        return pun_word, pun_sense, alter_word, alter_sense, context_words

    def extend_dict(given_dict, keys):
        # Create a null value based on the keys
        temp = given_dict
        for key in keys:
            if key not in temp:
                temp[key] = dict()
            temp = temp[key]
        return given_dict

    assert method in [0, 1, 2]
    path_gen = './results/' + path_gen
    if os.path.exists(path_gen):
        record_gen = load_json_file(path_gen)
    else:
        record_gen = dict()
    # [A]. Construct the prompt
    definition = """<*Definition*>\nPuns are a form of wordplay exploiting different meanings of a word or similar-sounding words, while non-puns are jokes or statements that don't rely on such linguistic ambiguities.\n\n"""
    # Method=0: Generate non-pun that only expresses one sense
    if method == 0:
        instruction = """<*Instruction*>\nBelow is a keyword and one of its meanings. Please generate a non-pun sentence with the keyword that conveys the given meaning. You must output the current status in a parsable JSON format. An example output looks like:\n{{"Sentence": "XXX"}}"""
        examples_string = """"""
        testing = """\n\n<*Your Response*>\nKeyword: {punchline}\nMeaning: {pun_word} <{pun_sense}>\nOutput:"""
    # Method=1: Generate pun directly
    elif method == 1:
        instruction = """<*Instruction*>\nBelow is a keyword and two of its meanings. Please generate a pun sentence with punchline on the keyword that conveys both given meanings simultaneously. Except for the keyword, the pun sentence must not utilize any words from either of the two meanings. Besides, once a keyword is used, it's strictly prohibited to use it again in the latter half of the sentence.\nYou must output the current status in a parsable JSON format. An example output looks like:\n{{"Sentence": "XXX"}}"""
        if examples is not None:
            examples_temp = []
            for ID in examples:
                example = examples[ID]
                pun_word, pun_sense, alter_word, alter_sense, context_words = get_punchline_elements(data=example)
                punchline, text = example['punchline'], example['human_text']
                examples_temp.append(f"Keyword: {punchline}\n"
                                     f"Meaning 1: {pun_word} <{pun_sense}>\n"
                                     f"Meaning 2: {alter_word} <{alter_sense}>\n"
                                     f"Output:\n{{{{\"Sentence\": \"{text}\"}}}}")
            examples_string = '\n\n<*Examples*>\n' + '\n\n'.join(examples_temp)
        else:
            examples_string = """"""
        testing = """\n\n<*Your Response*>\nKeyword: {punchline}\nMeaning 1: {pun_word} <{pun_sense}>\nMeaning 2: {alter_word} <{alter_sense}>\nOutput:"""
    # Method=2: Generate puns directly based on specific contextual words
    else:
        instruction = """<*Instruction*>\nBelow is a keyword, two of its meanings and a set of contextual words. Please generate a pun sentence with punchline on the keyword that conveys both given meanings simultaneously and using all the contextual words. Except for the keyword, the pun sentence must not utilize any words from either of the two meanings. Besides, once a keyword is used, it's strictly prohibited to use it again in the latter half of the sentence.\nYou must output the current status in a parsable JSON format. An example output looks like:\n{{"Sentence": "XXX"}}"""
        if examples is not None:
            examples_temp = []
            for ID in examples:
                example = examples[ID]
                pun_word, pun_sense, alter_word, alter_sense, context_words = get_punchline_elements(data=example)
                punchline, text = example['punchline'], example['human_text']
                examples_temp.append(f"Keyword: {punchline}\n"
                                     f"Meaning 1: {pun_word} <{pun_sense}>\n"
                                     f"Meaning 2: {alter_word} <{alter_sense}>\n"
                                     f"Contextual Words: {', '.join(context_words)}.\n"
                                     f"Output:\n{{{{\"Sentence\": \"{text}\"}}}}")
            examples_string = '\n\n<*Examples*>\n' + '\n\n'.join(examples_temp)
        else:
            examples_string = """"""
        testing = """\n\n<*Your Response*>\nKeyword: {punchline}\nMeaning 1: {pun_word} <{pun_sense}>\nMeaning 2: {alter_word} <{alter_sense}>\nContextual Words: {context_words}.\nOutput:"""
    # Combine all parts together
    prompt_string = definition + instruction + examples_string + testing

    # [B]. Call llm to generate
    key_gen = f"{model_name}_text"
    IDs = list(dataset.keys())
    IDs_loaded = []
    for ID in record_gen:
        if record_gen[ID].get(key_gen, False) and \
           record_gen[ID][key_gen].get(f'method {method}', False):
            IDs_loaded.append(ID)
    all_ind = list(range(0,len(IDs)))
    batch_ind = list(range(0,len(IDs),batch_size))
    for ind in tqdm(all_ind):
        if ind not in batch_ind:
            continue
        IDs_batch = IDs[ind: ind+batch_size]
        # Remove the data that has already been run
        IDs_batch = list(set(IDs_batch)-set(IDs_loaded))
        if len(IDs_batch) == 0:
            continue
        _inputs = []
        _human_text = []
        for ID in IDs_batch:
            data = dataset[ID]
            pun_ind = int(data['pun_word_ind'].split('_')[-1]) - 1
            punchline = data['human_text'].split(' ')[pun_ind]
            pun_word, pun_sense, alter_word, alter_sense, context_words = get_punchline_elements(data=data)
            _inputs.append(prompt_string.format(pun_word=pun_word, pun_sense=pun_sense,
                                                alter_word=alter_word, alter_sense=alter_sense,
                                                punchline=punchline, context_words=', '.join(context_words)))
            _human_text.append(data['human_text'])

        #_outputs = [out.content for out in model.batch(_inputs)]
        _outputs = []
        for i in range(len(_inputs)):
            print(f'case {ind}, {i}:')
            print(f'human text: {_human_text[i]}')
            if 'deepseek' in model_name:
                response = model.chat.completions.create(
                model="deepseek-chat",
                    messages=[
                        {"role": "user", "content": _inputs[i]},
                    ],
                    stream=False
                )
                
                #decode response JSON
                output = (json.loads(response.choices[0].message.content))['Sentence']
                print(f'LLM output: {output}')
                _outputs.append(output)
            elif 'qwen2' in model_name:
                url = "http://localhost:1234/v1/chat/completions"
                headers = {"Content-Type": "application/json"}
                payload = {
                    "model": model_name,
                    "messages": [
                        {"role": "user", "content": _inputs[i]},
                    ],
                    "temperature": 0.7,
                    "max_tokens": -1,
                    "stream": False
                }

                response = requests.post(url, headers=headers, data=json.dumps(payload))
                response = response.json()
                try:
                    output = json.loads(response['choices'][0]['message']['content'])['Sentence']
                except (KeyError, json.JSONDecodeError) as e:
                    print(f"Error parsing response: {e}")
                    output = response['choices'][0]['message']['content']
                print(f'LLM output: {output}')
                _outputs.append(output)
        
        # print(_inputs[0][0].content)
        # print(_outputs[0])
        # break
        for ID,out in zip(IDs_batch, _outputs):
            sentence = parse_output(ID, out)
            record_gen = extend_dict(record_gen, [ID, key_gen])
            record_gen[ID][key_gen].update({f'method {method}': sentence})
        if save:
            save_json_file(record_gen, path_gen)

## Dataset and Examples

In [88]:
hom_path = r'./dataset/hom_dataset.json'
het_path = r'./dataset/het_dataset.json'
hom_punDataset, hom_nonpunDataset = splitDataset(hom_path)
het_punDataset, het_nonpunDataset = splitDataset(het_path, puntype='het')

# add_human_pun(dict(**hom_punDataset, **het_punDataset), save=True)

In [89]:
# Choose data from examples manually
hom_examples = {
    "hom_705":{
        "pun_word": "toll",
        "pun_sense": "a fee levied for the use of roads or bridges (used for maintenance)",
        "alter_word": "toll",
        "alter_sense": "value measured by what must be given or done or undergone to obtain something",
        "punchline": "toll",
        "human_text":"Driving on so many turnpikes was taking its toll.",
        "human_keywords": ["Driving", "many", "turnpikes", "taking its toll"],
    },
    "hom_488":{
        "pun_word": "bore",
        "pun_sense": "make a hole, especially with a pointed power or hand tool",
        "alter_word": "bore",
        "alter_sense": "cause to be bored",
        "punchline": "bored",
        "human_text":"A carpenter sat on his drill and was bored to tears.",
        "human_keywords": ["carpenter", "sat", "drill", "bored to tears"],
    },
    "hom_1556":{
        "pun_word": "foil",
        "pun_sense": "a piece of thin and flexible sheet metal",
        "alter_word": "foil",
        "alter_sense": "hinder or prevent (the efforts, plans, or desires) of",
        "punchline": "foiled",
        "human_text":"One leftover said to another 'foiled again.'",
        "human_keywords": ["leftover", "foiled", "again"],
    }
}

het_examples = {
    "het_633": {
        "pun_word": "sagely",
        "pun_sense": "in a wise manner",
        "alter_word": "sage",
        "alter_sense": "aromatic fresh or dried grey-green leaves used widely as seasoning for meats and fowl and game etc",
        "punchline": "sagely",
        "human_text": "This fowl has been stuffed, said Tom sagely.",
        "human_keywords": ["fowl", "stuffed", "sagely"],
    },
    "het_530": {
        "pun_word": "toll",
        "pun_sense": "ring slowly",
        "alter_word": "tell off",
        "alter_sense": "reprimand",
        "punchline": "tolled",
        "human_text": "A tangled bell ringer tolled himself off.",
        "human_keywords": ["tangled", "bell ringer", "tolled himself off"],
    },
    "het_325": {
        "pun_word": "c",
        "pun_sense": "the 3rd letter of the Roman alphabet",
        "alter_word": "sea",
        "alter_sense": "a division of an ocean or a large body of salt water partially enclosed by land",
        "punchline": "c",
        "human_text": "An illiterate fisherman was lost at c.",
        "human_keywords": ["illiterate", "fisherman", "c"],
    }
}

## Generation

### DeepSeek-v3

In [90]:
# Connect DeepSeek-v3
deepseek_name = 'deepseek-chat'
temperature = 0.7
deepseek_api_key = get_deepseek_api_key()  # use your api key
deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")

In [95]:
# Generation method 2
call_llm_to_generate(model=deepseek, model_name=deepseek_name, dataset=hom_punDataset, method=2, examples=hom_examples, batch_size=10, save=True)
call_llm_to_generate(model=deepseek, model_name=deepseek_name, dataset=het_punDataset, method=2, examples=het_examples, batch_size=10, save=True)

  0%|          | 0/810 [00:00<?, ?it/s]

case 0, 0:
human text: Getting this job managing a country estate has put me off fried eggs . I ' m a gamekeeper turned poacher .
LLM output: The gamekeeper caught the fried eggs poacher.
case 0, 1:
human text: The intelligent entrepreneur ' s idea for designing catapults meant that his boss was completely thrown .


  0%|          | 0/810 [00:06<?, ?it/s]


KeyboardInterrupt: 

### Qwen2-vl-2b-instruct

In [96]:
qwen2vl = 'qwen2-vl-2b-instruct'
qwen2vl_name = 'qwen2-vl-2b-instruct'

In [103]:
# Generation method 2
call_llm_to_generate(model=qwen2vl, model_name=qwen2vl_name, dataset=hom_punDataset, method=2, examples=hom_examples, batch_size=10, save=True)
call_llm_to_generate(model=qwen2vl, model_name=qwen2vl_name, dataset=het_punDataset, method=2, examples=het_examples, batch_size=10, save=True)

  0%|          | 0/810 [00:00<?, ?it/s]

case 220, 0:
human text: Doctor , doctor , I keep thinking I ' m a deck of cards . - I ' ll deal with you later . Next !
LLM output: The doctor kept thinking 'I'm dealing with you later.'
case 220, 1:
human text: The ice at the rink has many ruts . I think the maintenance crew is slipping up .
LLM output: The maintenance crew slipped up on the slippery ice.
case 220, 2:
human text: Two companies that manufactured rulers decided to align .
LLM output: Companies have aligned themselves with new manufacturing methods and decided to align with the latest trends.
case 220, 3:
human text: Early nuclear experimenters discovered an element of surprise .
LLM output: Nuclear experimenters are looking for new elements.
case 220, 4:
human text: ' ' I ' ve run out of wool , ' ' said Tom , knitting his brow .
Error parsing response: Extra data: line 3 column 1 (char 69)
LLM output: Error: Unable to parse response : {e}
case 220, 5:
human text: To write with a broken pencil is pointless .
LLM output:

 27%|██▋       | 221/810 [00:12<00:32, 18.21it/s]

LLM output: I watched a man swallow his time and pass it back.
case 230, 0:
human text: Nobel got a big bang out of it .
LLM output: The Nobel had just given one last bang to the world.
case 230, 1:
human text: Doctor , Doctor , what would you take for this cold ? - Make me an offer . Next .
LLM output: The doctor made me an offer for the cold.
case 230, 2:
human text: ' ' Waiter , there ' s a fly in my soup ! ' ' ' ' Force of habit , sir , the chef used to be a tailor .
LLM output: The chef's tailors fly were flying again.
case 230, 3:
human text: Two pencils decided to have a race . The outcome was a draw .
LLM output: Two pencils drew two races, but the outcome was still uncertain.
case 230, 4:
human text: Those working in tissue research and testing are always blowing it .
LLM output: The test was blowing it.
case 230, 5:
human text: I crossed a cell phone with a skunk , and now the service stinks .
LLM output: This stinky service has crossed my cell phone.
case 230, 6:
human text:

 29%|██▊       | 231/810 [00:24<01:12,  7.94it/s]

...

LLM output: The businessmen must be feeling the inflation.
case 390, 0:
human text: He realized he ' d made the major leagues when it finally hit him .
LLM output: A player was hit by a pitch and left the game.
case 390, 1:
human text: My new theory on inertia doesn ' t seem to be gaining momentum .
LLM output: A theory was gaining momentum.
case 390, 2:
human text: I know a rancher who has 100 head of cattle , but he thought there were only 99 until he rounded them up .
LLM output: The rancher thought only of rounding them up.
case 390, 3:
human text: A locksmith has to have a good Combination of skills .
LLM output: The locksmith was able to combine his skills to solve the puzzle.
case 390, 4:
human text: Leonardo Da Vinci could use a paint brush to make a stroke of genius .
LLM output: The genius of Da Vinci had painted his stroke on the wall.
case 390, 5:
human text: OLD ACTUARIES never die they just get broken down by age and sex .
LLM output: An actuary died from the effects of b

 48%|████▊     | 391/810 [03:04<06:40,  1.05it/s]

LLM output: The arrows were quivering in the breeze.
case 400, 0:
human text: So , how ' s the flower business going ? Oh , it ' s blossoming quite nicely .
LLM output: The business is blossoming.
case 400, 1:
human text: Building a cul - de - sac would be a bit of a dead - end job .
LLM output: The cul-de-sac at the end of my driveway was a dead end to me.
case 400, 2:
human text: Swimming instructors are always getting immersed in their work .
LLM output: A swimming instructor immersed himself in the water and worked tirelessly.
case 400, 3:
human text: A dry cleaners had a dozen suits in court .
LLM output: The dry cleaner couldn't find a suit that fit me.
case 400, 4:
human text: My student was late for class , claiming he was in the washroom . I think he was stalling .
LLM output: The student claimed that he was in the washroom and might as well be in the stalling.
case 400, 5:
human text: We should make a beer commercial . It sounds simply intoxicating .
LLM output: Beer commerci

 49%|████▉     | 400/810 [03:12<03:17,  2.07it/s]


KeyboardInterrupt: 

### Qwen2-vl-2b-instruct

In [ ]:
qwen2vl = 'qwen2-vl-2b-instruct'
qwen2vl_name = 'qwen2-vl-2b-instruct'

In [ ]:
# Generation method 2
call_llm_to_generate(model=qwen2vl, model_name=qwen2vl_name, dataset=hom_punDataset, method=2, examples=hom_examples, batch_size=10, save=True)
call_llm_to_generate(model=qwen2vl, model_name=qwen2vl_name, dataset=het_punDataset, method=2, examples=het_examples, batch_size=10, save=True)

  0%|          | 0/810 [00:00<?, ?it/s]

case 220, 0:
human text: Doctor , doctor , I keep thinking I ' m a deck of cards . - I ' ll deal with you later . Next !
LLM output: The doctor kept thinking 'I'm dealing with you later.'
case 220, 1:
human text: The ice at the rink has many ruts . I think the maintenance crew is slipping up .
LLM output: The maintenance crew slipped up on the slippery ice.
case 220, 2:
human text: Two companies that manufactured rulers decided to align .
LLM output: Companies have aligned themselves with new manufacturing methods and decided to align with the latest trends.
case 220, 3:
human text: Early nuclear experimenters discovered an element of surprise .
LLM output: Nuclear experimenters are looking for new elements.
case 220, 4:
human text: ' ' I ' ve run out of wool , ' ' said Tom , knitting his brow .
Error parsing response: Extra data: line 3 column 1 (char 69)
LLM output: Error: Unable to parse response : {e}
case 220, 5:
human text: To write with a broken pencil is pointless .
LLM output:

 27%|██▋       | 221/810 [00:12<00:32, 18.21it/s]

LLM output: I watched a man swallow his time and pass it back.
case 230, 0:
human text: Nobel got a big bang out of it .
LLM output: The Nobel had just given one last bang to the world.
case 230, 1:
human text: Doctor , Doctor , what would you take for this cold ? - Make me an offer . Next .
LLM output: The doctor made me an offer for the cold.
case 230, 2:
human text: ' ' Waiter , there ' s a fly in my soup ! ' ' ' ' Force of habit , sir , the chef used to be a tailor .
LLM output: The chef's tailors fly were flying again.
case 230, 3:
human text: Two pencils decided to have a race . The outcome was a draw .
LLM output: Two pencils drew two races, but the outcome was still uncertain.
case 230, 4:
human text: Those working in tissue research and testing are always blowing it .
LLM output: The test was blowing it.
case 230, 5:
human text: I crossed a cell phone with a skunk , and now the service stinks .
LLM output: This stinky service has crossed my cell phone.
case 230, 6:
human text:

 29%|██▊       | 231/810 [00:24<01:12,  7.94it/s]

...

LLM output: The businessmen must be feeling the inflation.
case 390, 0:
human text: He realized he ' d made the major leagues when it finally hit him .
LLM output: A player was hit by a pitch and left the game.
case 390, 1:
human text: My new theory on inertia doesn ' t seem to be gaining momentum .
LLM output: A theory was gaining momentum.
case 390, 2:
human text: I know a rancher who has 100 head of cattle , but he thought there were only 99 until he rounded them up .
LLM output: The rancher thought only of rounding them up.
case 390, 3:
human text: A locksmith has to have a good Combination of skills .
LLM output: The locksmith was able to combine his skills to solve the puzzle.
case 390, 4:
human text: Leonardo Da Vinci could use a paint brush to make a stroke of genius .
LLM output: The genius of Da Vinci had painted his stroke on the wall.
case 390, 5:
human text: OLD ACTUARIES never die they just get broken down by age and sex .
LLM output: An actuary died from the effects of b

 48%|████▊     | 391/810 [03:04<06:40,  1.05it/s]

LLM output: The arrows were quivering in the breeze.
case 400, 0:
human text: So , how ' s the flower business going ? Oh , it ' s blossoming quite nicely .
LLM output: The business is blossoming.
case 400, 1:
human text: Building a cul - de - sac would be a bit of a dead - end job .
LLM output: The cul-de-sac at the end of my driveway was a dead end to me.
case 400, 2:
human text: Swimming instructors are always getting immersed in their work .
LLM output: A swimming instructor immersed himself in the water and worked tirelessly.
case 400, 3:
human text: A dry cleaners had a dozen suits in court .
LLM output: The dry cleaner couldn't find a suit that fit me.
case 400, 4:
human text: My student was late for class , claiming he was in the washroom . I think he was stalling .
LLM output: The student claimed that he was in the washroom and might as well be in the stalling.
case 400, 5:
human text: We should make a beer commercial . It sounds simply intoxicating .
LLM output: Beer commerci

 49%|████▉     | 400/810 [03:12<03:17,  2.07it/s]


KeyboardInterrupt: 